In [1]:
from mpl_toolkits.basemap import Basemap  # import Basemap matplotlib toolkit
import numpy as np
import matplotlib.pyplot as plt
import pygrib # import pygrib interface to grib_api
import pandas as pd
import os
import sys
import math
import folium
import csv 
import time

start = time.time()

%matplotlib inline

root = os.getcwd()
root_data = root + "/grib_data"
root_URL = "http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/"

#timestomp like 201808010000 means that it is at 2019/04/09/08:00(UTC timezone)
year = "2019"
month = "04"
day = "22"
roottime="000000"
FH="00-15"

def getgrib(year,month,day,roottime,FH):
    file_name = "Z__C_RJTD_"+str(year+month+day+roottime)+"_MSM_GPV_Rjp_Lsurf_FH"+str(FH)+"_grib2.bin"
    file_url = root_URL\
                +str(year + "/" + month + "/" + day +"/")\
                +file_name
    root_file= "grib_data/"+file_name
    if os.path.isfile(root_file) == False:
            command = "wget -P grib_data/ "+ file_url
            os.system(command)

    getgrib_output = pygrib.open(root_file)
    return getgrib_output

def getrootgrib(year,month,day,time,FH):
    file_name = "Z__C_RJTD_"+str(year+month+day+roottime)+"_MSM_GPV_Rjp_Lsurf_FH"+str(FH)+"_grib2.bin"   
    return "grib_data/"+file_name

In [2]:
print(os.path.isfile( getrootgrib(year,month,day,roottime,FH) ))

True


In [3]:
grbs = getgrib(year,month,day,roottime,FH)
latlons = grbs

In [4]:
print(os.path.isfile( getrootgrib(year,month,day,roottime,FH) ))

True


In [5]:
#check parameters in gribfile
grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb_1.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_rad= {'lons':flat_lons, 'lats':flat_lats}
pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
pd_temp= {'lons':flat_lons, 'lats':flat_lats}

i,j,k,l =0,0,0,0

for grb in grbs:
    print(grb)
    
    if grb.name == "Downward short-wave radiation flux":
        radiation = grb.values
        flat_radiation = np.ravel(radiation)
        pd_name="radiation FH"+str(i)+"-"+str(i+1)+"W/m2(avg)"
        pd_rad[pd_name]=flat_radiation
        i=i+1

    elif grb.name == "10 metre U wind component":
        Uwind = grb.values
        flat_Uwind = np.ravel(Uwind)
        pd_name="Uwind FH"+str(j)+"m/s"
        pd_Uwind[pd_name]=flat_Uwind
        j=j+1

    elif grb.name == "10 metre V wind component":
        Vwind = grb.values
        flat_Vwind = np.ravel(Vwind)
        pd_name="Vwind FH"+str(k)+"m/s"
        pd_Vwind[pd_name]=flat_Vwind
        k=k+1

    elif grb.name == "Temperature":
        temperature = grb.values
        #華氏を摂氏に変換
        flat_temperature= np.ravel(temperature)
        flat_temperature=flat_temperature-273.115
        pd_name="temperature FH"+str(l)
        pd_temp[pd_name]=flat_temperature
        l=l+1

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 0 hrs:from 201904220000
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904220000
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201904220000
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201904220000
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201904220000
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201904220000
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904220000
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904220000
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904220000
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 2

110:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 9 hrs:from 201904220000
111:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 9 hrs:from 201904220000
112:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 9 hrs:from 201904220000
113:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201904220000
114:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201904220000
115:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201904220000
116:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 9 hrs:from 201904220000
117:Total precipitation:kg m-2 (accum):regular_ll:surface:level 0:fcst time 8-9 hrs (accum):from 201904220000
118:Downward short-wave radiation flux:W m**-2 (avg):regular_ll:surface:level 0:fcst time 8-9 hrs (avg):from 201904220000
119:Pressure reduced to MSL:Pa (instant):regular_ll:mean

In [6]:
#緯度経度の範囲を選択
east_lon=130.438
west_lon= 130.437
north_lat=33.555
south_lat=33.550

In [7]:
#range adjust
def range_ajust(east_lon,west_lon,north_lat,south_lat,df_all):
    df=df_all[df_all['lons'] > west_lon]
    df=df[df['lons'] < east_lon]

    df=df[df['lats'] > south_lat]
    df=df[df['lats'] < north_lat]
    return df



## Select parameter temperature, Uwind, Vwind, radiation

In [8]:
########################
df_para = "Uwind"
########################

if df_para == "temperature":
    df_input=pd.DataFrame(pd_temp)

elif df_para == "Uwind":
    df_input=pd.DataFrame(pd_Uwind)
   
elif df_para == "Vwind":
    df_input=pd.DataFrame(pd_Vwind)
    
elif df_para == "radiation":
    df_input=pd.DataFrame(pd_rad)
    


df = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input)
map_lat=df['lats'].values
map_lon=df['lons'].values
df = np.round(df,2)
print(len(df))
df[:10]

1


,lons,lats,Uwind FH0m/s,Uwind FH1m/s,Uwind FH2m/s,Uwind FH3m/s,Uwind FH4m/s,Uwind FH5m/s,Uwind FH6m/s,Uwind FH7m/s,Uwind FH8m/s,Uwind FH9m/s,Uwind FH10m/s,Uwind FH11m/s,Uwind FH12m/s,Uwind FH13m/s,Uwind FH14m/s,Uwind FH15m/s
135328,130.44,33.55,0.12,-0.07,0.43,1.17,1.28,0.83,0.92,1.01,1.26,1.62,1.41,1.19,0.94,0.51,0.17,-0.21


In [9]:
FH0=df.iloc[:,2].values
FH1=df.iloc[:,3].values
FH2=df.iloc[:,4].values
FH3=df.iloc[:,5].values
FH4=df.iloc[:,6].values
FH5=df.iloc[:,7].values
FH6=df.iloc[:,8].values
FH7=df.iloc[:,9].values
FH8=df.iloc[:,10].values
FH9=df.iloc[:,11].values
FH10=df.iloc[:,12].values
FH11=df.iloc[:,13].values
FH12=df.iloc[:,14].values
FH13=df.iloc[:,15].values
FH14=df.iloc[:,16].values
FH15=df.iloc[:,17].values


In [10]:
from folium import plugins

#lat-long of focus area and zoom start
map = folium.Map([33.518022,130.471583], zoom_start=12)

# Mapbox makes some nice map tiles,
# Stamen also produce some cool map tiles which typically work at all zoom levels.
tile = folium.TileLayer('Stamen Terrain').add_to(map)

#adding marker and popup of temperature
for i in range(0,len(map_lat)):
    
    icon_number = plugins.BeautifyIcon(
    border_color='#00ABDC',
    text_color='#00ABDC',
    border_width=2,
    number=str(int(FH0[i])),
    inner_icon_style='margin-top:0;')
    
    folium.Marker(location=[float(map_lat[i])  ,float(map_lon[i])],\
                  popup="___"+df_para+"___"+"\n"\
                          +"__lat= "+str(map_lat[i])+"__\n"\
                          +"__lon= "+str(map_lon[i])+"__\n"\
                          +"__FH0 = "+str(FH0[i])+"__\n"\
                          +"__FH1 = "+str(FH1[i])+"__\n"\
                          +"__FH2 = "+str(FH2[i])+"__\n"\
                          +"__FH3 = "+str(FH3[i])+"__\n"\
                          +"__FH4 = "+str(FH4[i])+"__\n"\
                          +"__FH5 = "+str(FH5[i])+"__\n"\
                          +"__FH6 = "+str(FH6[i])+"__\n"\
                          +"__FH7 = "+str(FH7[i])+"__\n"\
                          +"__FH8 = "+str(FH8[i])+"__\n"\
                          +"__FH9 = "+str(FH9[i])+"__\n"\
                          +"__FH10 = "+str(FH10[i])+"__\n"\
                          +"__FH11 = "+str(FH11[i])+"__\n"\
                          +"__FH12 = "+str(FH12[i])+"__\n"\
                          +"__FH13 = "+str(FH13[i])+"__\n"\
                          +"__FH14 = "+str(FH14[i])+"__\n"\
                          #+"__FH15 = "+str(FH15[i])+"__\n"\
                          ,\
                 icon=icon_number).add_to(map)
    
#we can change tiles with help of LayerConrol
folium.LayerControl().add_to(map)

#Adding mouse position
formatter = "function(num) {return L.Util.formatNum(num, 3) + ' º ';};"

folium.plugins.MousePosition(
    position='topright',
    separator=' | ',
    empty_string='NaN',
    lng_first=True,
    num_digits=20,
    prefix='Coordinates:',
    lat_formatter=formatter,
    lng_formatter=formatter
).add_to(map)

map_name="Output_MAP/"+"FH_"+FH+"_"+df_para+"_Map_" + year + "_" + month + "_" + day+ "_" + roottime+".html"
csv_name="Output_CSV/"+"FH_"+FH+"_"+df_para + year + "_" + month + "_" + day+ "_" + roottime+".csv"

df.to_csv(csv_name, encoding='utf-8', index=False)
map.save(outfile=map_name)

map

In [11]:
elapsed_time = time.time() - start

print("elapsed_time:{0}".format(elapsed_time))

elapsed_time:1.717902660369873
